# Natural Language Processing - Assigment 2
# N-gram Language Models

In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this assignment, we will focus on the related problem of predicting the next *character* or *word* in a sequence given the previous characters. 

The learning goals of this assignment are to: 
* Understand how to compute language model probabilities using maximum likelihood estimation.
* Implement basic smoothing, back-off and interpolation.
* Have fun using a language model to probabilistically generate texts.
* Compare word-level langauage models and character-level language models

<div class="alert alert-info" markdown="1">
Here are the materials that you should download for this assignment:

* [Skeleton python code](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/skeleton.zip).

* [training data for character-level and word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/shakespeare_input.zip).

* [dev data for character-level and word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/shakespeare_sonnets.zip).

* [training data for word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/train_e.zip).

* [dev data for word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/val_e.zip).
</div>


# Character-level N-gram Language Models [20 pts]

You should complete functions in the script `hw2_skeleton_char.py` in this part. After submitting `hw2_skeleton_char.py` to Gradescope and passing all the test cases for this part, you can get full score.

## Part 1: Generation

In [3]:
%load_ext autoreload
%autoreload 2
from skeleton.hw2_skeleton_char import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random

Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the $n$ characters preceding the current character. The sentence should be padded with $n$ ~ characters at the beginning (we've provided you with `start_pad(n)` for this purpose). If $n=0$, all contexts should be empty strings. You may assume that $n\ge0$.

In [3]:
ngrams(2, 'abc')

[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]

We've also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided. You should use it.

You will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, NumPy and NLTK.

1. In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order $n$ of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the set of all characters used by this model).

2. Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be $1/V$ where $V$ is the size of the vocab.

3. Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $0\le r<1$ be a random number between 0 and 1. Your method should return the character $v_i$ such that

    $$\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).$$

    You should use a single call to the `random.random()` function to generate $r$.

In [4]:
import math
math.pow(2, 3)

8.0

In [5]:
m = NgramModel(1, 0)

In [6]:
m.update('abab')
m.get_vocab()

{'a', 'b'}

In [7]:
m.update('abcd')
m.get_vocab()

{'a', 'b', 'c', 'd'}

In [8]:
m.prob('a', 'b')

1.0

In [9]:
m.prob('~', 'c')

0.0

In [10]:
m.prob('b', 'c')

0.5

In [11]:
m.update('abab')
m.update('abcd')

In [12]:
# random.seed(1)
sorted([m.random_char('b') for i in range(25)])

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c']

In [13]:
# random.seed(1)
m.random_char('abcd')

'b'

4. In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

In [14]:
from skeleton.hw2_skeleton_char import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
m = NgramModel(1, 0.1)
m.update('abab')
m.update('abcd')
random.seed(1)
m.random_text(25)

'abcdbabcbabababcdddaabcda'

### Writing Shakespeare 

Now you can train a language model. First grab some text like [this corpus of Shakespeare](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/shakespeare_input.zip).

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

In [15]:
random.seed(1)

In [47]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
m.random_text(250)

"Fir of yould brin most some a wou to gety.\n\nShavere laguis prinsaved,\nAEGARMACURESS Andeavelf it teronot wromper'd swe sur\nOphy, sh tholovence er hisoleasou aing forrat struchis shost of you are rock a be mothatilly shus,\n'Twom,\nCHARUTUS:\nAbou, now'd"

In [48]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
m.random_text(250)

"First\nthe deliege\n'Twas empers.\n\nAlarent\nCall no heave gents a gentlemean he a rus man.\n\nIAGO:\nUnless, woul finding why he powere to to my form brothat face,\nThen, there's a good\nrevert oft in\nand dumpetire.\n\nTAMORENCE:\nYou speak wildeny you\nsee daug"

In [49]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
m.random_text(250)

'First Service;\nFor me awhilst.\n\nSEBASTINGS:\nHer be the true, when\nAmbitious and followers, none.\nA sing stubborn imprisoner. Ford; Mistress, where:\nShe action! Belaribel touch is but thence\nAnd wants: have thou say, ever did, Lord Bardon myself the c'

In [41]:
# Find the reason why it is started with 'F'
m.ngrams_char_count
m.prob('~~~~','F')

1.0

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? First, generate the word *First*. Explain what is going on in your writeup.

## Part 2: Perplexity, Smoothing, and Interpolation

In this part of the assignment, you'll adapt your code in order to implement several of the  techniques described in [Section 3 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

### Perplexity

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as extrinsic evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. Intrinsic evaluation. Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $$W = w_1 w_2 ... w_N$$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.

We provide you [dev data for character-level and word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/shakespeare_sonnets.zip).


In [19]:
float('inf')

inf

In [20]:
from skeleton.hw2_skeleton_char import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
m = NgramModel(1, 0)
m.update('abcd')
m.update('acbd')
m.perplexity('acd')

1.5874010519681994

In [21]:
m.perplexity('abca')

inf

In [22]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, k=1)
with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

7.910100720888779


Note: you may want to create a smoothed language model before calculating perplexity on real data.

### Smoothing

Laplace Smoothing is described in section 4.4.1. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section Add-k 4.4.2. Update your `NgramModel` code from Part 1 to implement add-k smoothing.

In [23]:

m = NgramModel(1, 1)
m.update('abab')
m.update('abcd')
m.prob('a', 'a')

0.14285714285714285

In [24]:
m.perplexity('abca')

2.691781635477648

In [25]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

67
7.996946415762477


### Interpolation

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

In [1]:
from skeleton.hw2_skeleton_char import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
m = NgramModelWithInterpolation(2, 0)

m.update('abab')
m.update('abcd')
print(m.ngrams)
print('---')
print(m.ngrams_char_count)
m.prob('ab', 'a')


{'': 8, '~': 2, 'a': 3, 'b': 2, '~~': 2, '~a': 2, 'ab': 2, 'ba': 1, 'c': 1, 'bc': 1}
---
{'': {'a': 3, 'b': 3, 'c': 1, 'd': 1}, '~': {'a': 2}, 'a': {'b': 3}, 'b': {'a': 1, 'c': 1}, '~~': {'a': 2}, '~a': {'b': 2}, 'ab': {'a': 1, 'c': 1}, 'ba': {'b': 1}, 'c': {'d': 1}, 'bc': {'d': 1}}


0.4583333333333333

In [14]:
m.set_lambda([1/3, 1/3, 1/3])
m.perplexity('abca')

[1.0, 1.0, 0.375]
[0.3333333333333333, 0.3333333333333333, 0.125]
[1.0, 1.0, 0.375]
[0.3333333333333333, 0.3333333333333333, 0.125]
[0.5, 0.5, 0.125]
[0.16666666666666666, 0.16666666666666666, 0.041666666666666664]
[0.0, 0.0, 0.375]
[0.0, 0.0, 0.125]


2.4154246840757705

In [3]:
m = NgramModelWithInterpolation(2, 1)
m.update('abab')
m.update('abcd')
m.prob('~a', 'b')

[0.5, 0.5714285714285714, 0.3333333333333333]
[0.16666666666666666, 0.19047619047619047, 0.1111111111111111]


0.4682539682539682

In [29]:
m.perplexity('abca')

2.9003863011784152

In [2]:
n_list = [1,2,3,4]
k_list = [0.01, 0.05, 0.1, 0.5]
lamb_dict = dict()
lamb_dict[1] = [[1,0], [0.8,0.2], [0.5,0.5], [0.2,0.8]]
lamb_dict[2] = [[1,0,0], [0.8,0.1,0.1], [0.5,0.25,0.25], [1/3,1/3,1/3]]
lamb_dict[3] = [[1,0,0,0], [0.7,0.1,0.1,0.1], [0.5, 0.5/3, 0.5/3, 0.5/3], [1/4,1/4,1/4,1/4]]
lamb_dict[4] = [[1,0,0,0,0], [0.6,0.1,0.1,0.1,0.1], [0.6,0.2,0.1,0.1,0], [0.5,0.2,0.2,0.1,0]]
for n in n_list:
    for k in k_list:
        for lamb_list in lamb_dict[n]:
            m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)
            m.set_lambda(lamb_list)
            print('======')
            print('k is ', k)
            print('n is ', n)
            print('lambda is ', lamb_list)
            with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
                print(m.perplexity(f.read()))

k is  0.01
n is  1
lambda is  [1, 0]
12.507064670984176
k is  0.01
n is  1
lambda is  [0.8, 0.2]
12.377156380683624
k is  0.01
n is  1
lambda is  [0.5, 0.5]
14.268139831989634
k is  0.01
n is  1
lambda is  [0.2, 0.8]
17.928968715826137
k is  0.05
n is  1
lambda is  [1, 0]
12.414256866324168
k is  0.05
n is  1
lambda is  [0.8, 0.2]
12.375785519209728
k is  0.05
n is  1
lambda is  [0.5, 0.5]
14.267556112464746
k is  0.05
n is  1
lambda is  [0.2, 0.8]
17.928824714810624
k is  0.1
n is  1
lambda is  [1, 0]
12.374521854308794
k is  0.1
n is  1
lambda is  [0.8, 0.2]
12.375152761290542
k is  0.1
n is  1
lambda is  [0.5, 0.5]
14.267369215374984
k is  0.1
n is  1
lambda is  [0.2, 0.8]
17.92887276653698
k is  0.5
n is  1
lambda is  [1, 0]
12.281936233797339
k is  0.5
n is  1
lambda is  [0.8, 0.2]
12.374048269480218
k is  0.5
n is  1
lambda is  [0.5, 0.5]
14.267795953956414
k is  0.5
n is  1
lambda is  [0.2, 0.8]
17.92990654314159
k is  0.01
n is  2
lambda is  [1, 0, 0]
8.168500693819311
k is  0.

In [6]:
from skeleton.hw2_skeleton_char import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
n_list = [5]
k_list = [0.0001, 0.0005, 0.01]
lamb_dict = dict()
lamb_dict[5] = [[0.5,0.2,0.2,0.05,0.05,0], [0.4, 0.3,0.2,0.05,0.05,0] , [0.3,0.3,0.2,0.15,0.05,0]]
for n in n_list:
    for k in k_list:
        for lamb_list in lamb_dict[n]:
            m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)
            m.set_lambda(lamb_list)
            print('======')
            print('k is ', k)
            print('n is ', n)
            print('lambda is ', lamb_list)
            with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
                print(m.perplexity(f.read()))

k is  0.0001
n is  5
lambda is  [0.5, 0.2, 0.2, 0.05, 0.05, 0]
5.180455531950972
k is  0.0001
n is  5
lambda is  [0.4, 0.3, 0.2, 0.05, 0.05, 0]
5.171307105538554
k is  0.0001
n is  5
lambda is  [0.3, 0.3, 0.2, 0.15, 0.05, 0]
5.253177336604155
k is  0.0005
n is  5
lambda is  [0.5, 0.2, 0.2, 0.05, 0.05, 0]
5.141533203865685
k is  0.0005
n is  5
lambda is  [0.4, 0.3, 0.2, 0.05, 0.05, 0]
5.132534771174637
k is  0.0005
n is  5
lambda is  [0.3, 0.3, 0.2, 0.15, 0.05, 0]
5.214595687678927
k is  0.01
n is  5
lambda is  [0.5, 0.2, 0.2, 0.05, 0.05, 0]
5.112652121036909
k is  0.01
n is  5
lambda is  [0.4, 0.3, 0.2, 0.05, 0.05, 0]
5.0998365326022626
k is  0.01
n is  5
lambda is  [0.3, 0.3, 0.2, 0.15, 0.05, 0]
5.180063861930196


In [6]:
n_list = [6]
k_list = [0.01, 0.05, 0.1, 0.5]
lamb_dict = dict()
lamb_dict[6] = [[0.3, 0.2,0.15,0.15,0.1,0.1,0], [0.4, 0.3,0.1, 0.1,0.05,0.05,0]]
for n in n_list:
    for k in k_list:
        for lamb_list in lamb_dict[n]:
            m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)
            m.set_lambda(lamb_list)
            print('======')
            print('k is ', k)
            print('n is ', n)
            print('lambda is ', lamb_list)
            with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
                print(m.perplexity(f.read()))

k is  0.01
n is  6
lambda is  [0.3, 0.2, 0.15, 0.15, 0.1, 0.1, 0]
5.249738443251137
k is  0.01
n is  6
lambda is  [0.4, 0.3, 0.1, 0.1, 0.05, 0.05, 0]
5.252540350332047
k is  0.05
n is  6
lambda is  [0.3, 0.2, 0.15, 0.15, 0.1, 0.1, 0]
5.491573482348842
k is  0.05
n is  6
lambda is  [0.4, 0.3, 0.1, 0.1, 0.05, 0.05, 0]
5.544356447189646
k is  0.1
n is  6
lambda is  [0.3, 0.2, 0.15, 0.15, 0.1, 0.1, 0]
5.71220073819207
k is  0.1
n is  6
lambda is  [0.4, 0.3, 0.1, 0.1, 0.05, 0.05, 0]
5.822176043580539
k is  0.5
n is  6
lambda is  [0.3, 0.2, 0.15, 0.15, 0.1, 0.1, 0]
6.658200394395542
k is  0.5
n is  6
lambda is  [0.4, 0.3, 0.1, 0.1, 0.05, 0.05, 0]
7.073804333274236


In your report, experiment with a few different lambdas and values of k and discuss their effects.

# Word-level N-gram Language Models [Required for CS 7650. Bonus for CS 4650] [10 pts]

You should complete functions in the script `hw2_skeleton_word.py` in this part. After submitting `hw2_skeleton_word.py` to Gradescope and passing all the test cases for this part, you can get full score. Instructions are similar to the instructions above. It is convenient to first use `text.strip().split()` to convert a string of word sequence to a list of words. In some functions, we provide `text.strip().split()`. You can use it optionally. 

Besides the corpus above, we also provide you [training data for word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/train_e.zip) and [dev data for word-level langauge models](https://www.cc.gatech.edu/classes/AY2021/cs7650_fall/programming/HW2/val_e.zip), in which each sentence has been processed with word tokenizer and `[EOS]` token has been appended to the end of each sentences.  `[EOS]` can be regarded as the sentence boundary when generating a paragraph or evaluating the perplexity of a paragraph.


## Part 1: Generation

In [7]:
from skeleton.hw2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random

In [32]:
ngrams(2, 'I love Natural Language Processing')

[('~ ~', 'I'),
 ('~ I', 'love'),
 ('I love', 'Natural'),
 ('love Natural', 'Language'),
 ('Natural Language', 'Processing')]

In [33]:
m = NgramModel(1, 0.1)

In [13]:
m.update('I love natural language processing')
m.get_vocab()

{'i', 'language', 'love', 'natural', 'processing'}

In [14]:
m.update('I love machine learning')
m.get_vocab()
print(m.ngrams)
print('===')
print(m.ngrams_char_count)

{'~': 2, 'i': 2, 'love': 2, 'natural': 1, 'language': 1, 'machine': 1}
===
{'~': {'i': 2}, 'i': {'love': 2}, 'love': {'natural': 1, 'machine': 1}, 'natural': {'language': 1}, 'language': {'processing': 1}, 'machine': {'learning': 1}}


In [15]:
m.prob('I', 'love')

0.14285714285714285

In [16]:
m.prob('~', 'You')

0.037037037037037035

In [17]:
m.prob('love', 'natural')

0.40740740740740744

In [18]:
m.update('You love computer vision')
m.update('I was late today')

In [19]:
random.seed(1)
[m.random_word('~') for i in range(25)]

['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision', 'was', 'you']
['computer', 'i', 'language', 'late', 'learning', 'love', 'machine', 'natural', 'processing', 'today', 'vision'

['i',
 'you',
 'vision',
 'i',
 'i',
 'i',
 'learning',
 'was',
 'i',
 'i',
 'you',
 'i',
 'vision',
 'computer',
 'i',
 'processing',
 'i',
 'you',
 'you',
 'i',
 'i',
 'i',
 'you',
 'i',
 'i']

In [27]:
m = NgramModel(1, 0)
m.update('You are welcome')
m.update('We are friends')
random.seed(1)
m.random_text(25)
''.join(' '.join(['~'] * 2).split(' ')[1:]) + ' love'

['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'friends', 'we', 'welcome', 'you']
['are', 'fr

'~ love'

In [45]:
m = create_ngram_model(NgramModel, 'train_e.txt', 2)
m.random_text(250)


"In the psychodynamic therapy truism , mothers of only 12 years . [E0S] It was a `` level the scores level and point guard to find out for `` travel expenses . [E0S] Dressed in their area , '' Kiraly said . [E0S] 1am : 'Female victim : the company released a report from a manhole cover , where we have questions for U.S. forces in Wales [E0S] Levels of acetone , and cared for thousands of former students , especially now that her research on children 's favourite recipe had to employ staff who started the second embarrassing discovery late in the autumn . [E0S] Any local council has said virtual reality , go on strike in their tricky relationship . [E0S] But while Mel McLaughlin . [E0S] I think that after a furious response , he noted the `` great nation , and Zita , two ; with the step up all their Christmases were ruined by Vanessa 's little sister . [E0S] As the years [E0S] Selfie : But let 's rest and make socialising the focus is a problem and not let Corbyn 'experiment ' with her 

In [13]:
m = create_ngram_model(NgramModel, 'train_e.txt', 3)
m.random_text(250)

"In the immediate term , though , with Horlin-Smith beating the brothers and uploading them to YouTube . [E0S] Yet many researchers show us that this `` most virtuous and wise '' son of the Republican candidates are all promising they would do otherwise . [E0S] Another couple celebrating graduation would certainly make a memorable couple if their faces were true to life [E0S] The porcelain girl has been missing from north London said she did not show up at the mercy of shysters in places you do n't want to hear from the president , `` Do I not hate those who hate the West and its allies are inciting proliferation and giving moral and legal cover to the likes of Donald Trump . [E0S] With a record jackpot on offer , `` they would be reluctant to negotiate anything major with a lame duck could shovel a lot of talk about the next iPhone is released , according to the criminal complaint said . [E0S] The scoop in this image taken on June 23 . [E0S] Here 's what you 're having children '' [E0

In [14]:
m = create_ngram_model(NgramModel, 'train_e.txt', 4)
m.random_text(250)

"In the immediate term , though , that lurk in the pitch , particularly against the new ball . [E0S] Oscar nominee Ellen Page ( `` Juno '' ) can be seen warning that the West will eventually turn on its Muslim population , before Trump is heard calling for the ban [E0S] The front-runner is Democrat Joaquin Arambula , an emergency room physician whose father once was an assemblyman . [E0S] Popeye is a popular figure with the players and apparently he and his wife Kim do not know the sex of their baby but have revealed that it is due in July . [E0S] And what 's the number one life lesson you want people , when they sing , they do n't care how much noise they make . [E0S] Now Playing : 'GMA ' Ultimate Tailgating Challenge ' : Lara 's Nittany Nachos [E0S] Suu Kyi 's party prepares to take charge of Myanmar parliament [E0S] The Nerang 'Loch Ness monster '' snake.Source : Supplied [E0S] A soldier from India 's counter-terrorism force , the National Security Guard ( NSG ) led the fight agains

Do you think these outputs are more reasonable than character-level language models?

After generating a bunch of short passages, do you notice anything? *They all start with In!*  Why is that? Is the model trying to be clever? First, generate the word *In*. Explain what is going on in your writeup.

## Part 2: Perplexity, Smoothing, and Interpolation

### Perplexity

In [51]:
from skeleton.hw2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
m = NgramModel(1, 0.1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.perplexity('I love machine learning')

2.0409040291494223

In [52]:
m.perplexity('I love python')

4.885785532341627

In [53]:
m = create_ngram_model(NgramModel, 'train_e.txt', 2, k=0.1)
with open('val_e.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

30617.796152024268


### Smoothing

In [8]:
m = NgramModel(1, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.perplexity('I love machine learning')

4.743416490252569

In [9]:
m.perplexity('I love python')

6.0822019955734

In [10]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

62983
51817.86687395116


In [11]:
m = create_ngram_model(NgramModel, 'train_e.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('val_e.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

129555
30617.796152024268


### Interpolation

In [1]:
from skeleton.hw2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
m = NgramModelWithInterpolation(2, 0)
m.update('I love natural language processing')
m.update('You love machine learning')
m.prob('I love','machine')

[0.0, 0.5, 0.1111111111111111]


0.20370370370370372

In [2]:
m.perplexity('I love machine learning')
a = {'acc', 'ba','a'}
sorted(list(a))

[0.5, 0.5, 0.1111111111111111]
[1.0, 1.0, 0.2222222222222222]
[0.0, 0.5, 0.1111111111111111]
[1.0, 1.0, 0.1111111111111111]


['a', 'acc', 'ba']

In [3]:
m = NgramModelWithInterpolation(2, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.prob('~ I','love')

0.16623093681917211

In [4]:
m.perplexity('I love machine learning')

6.76855019085475

In [2]:
from skeleton.hw2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
n_list = [1]
k_list = [0.01, 0.001, 0.005]
lamb_dict = dict()
lamb_dict[1] = [[0.6,0.4], [0.5,0.5], [0.4,0.6]]
# lamb_dict[2] = [[1,0,0], [0.8,0.1,0.1], [0.5,0.25,0.25], [1/3,1/3,1/3]]
# lamb_dict[3] = [[1,0,0,0], [0.7,0.1,0.1,0.1], [0.5, 0.5/3, 0.5/3, 0.5/3], [1/4,1/4,1/4,1/4]]
# lamb_dict[4] = [[1,0,0,0,0], [0.6,0.1,0.1,0.1,0.1], [0.6,0.2,0.1,0.1,0], [0.5,0.2,0.2,0.1,0]]
for n in n_list:
    for k in k_list:
        for lamb_list in lamb_dict[n]:
            m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n, k)
            m.set_lambda(lamb_list)
            print('======')
            print('k is ', k)
            print('n is ', n)
            print('lambda is ', lamb_list)
            with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
                print(m.perplexity(f.read()))
# m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', 3, k=0.5)
# m.lamb=[0.3,0.2,0.2,0.3]
# print(len(m.get_vocab()))
# with open('shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
#     print(m.perplexity(f.read()))

k is  0.01
n is  1
lambda is  [0.6, 0.4]
3181.708038897317
k is  0.01
n is  1
lambda is  [0.5, 0.5]
3079.893266034102
k is  0.01
n is  1
lambda is  [0.4, 0.6]
3050.561110210501
k is  0.001
n is  1
lambda is  [0.6, 0.4]
3184.4211792037318
k is  0.001
n is  1
lambda is  [0.5, 0.5]
3090.8575120741034
k is  0.001
n is  1
lambda is  [0.4, 0.6]
3077.52730372507
k is  0.005
n is  1
lambda is  [0.6, 0.4]
3130.2175852619416
k is  0.005
n is  1
lambda is  [0.5, 0.5]
3037.3760614840326
k is  0.005
n is  1
lambda is  [0.4, 0.6]
3017.7192982324673


Running the following code could take about 10 minutes. This should be finished within 15 minutes.

In [5]:
from skeleton.hw2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random
n_list = [1,2]
# k_list = [0.01, 0.05, 0.1, 0.5]
lamb_dict = dict()
# lamb_dict[1] = [[1,0], [0.8,0.2], [0.5,0.5], [0.2,0.8]]
lamb_dict[2] = [[0.8,0.1,0.1], [0.5,0.25,0.25], [1/3,1/3,1/3], [0.2, 0.5,0.3]]
# lamb_dict[3] = [[1,0,0,0], [0.7,0.1,0.1,0.1], [0.5, 0.5/3, 0.5/3, 0.5/3], [1/4,1/4,1/4,1/4]]
# lamb_dict[4] = [[1,0,0,0,0], [0.6,0.1,0.1,0.1,0.1], [0.6,0.2,0.1,0.1,0], [0.5,0.2,0.2,0.1,0]]
# n_list = [1]
k_list = [0.01, 0.001, 0.005]
# lamb_dict = dict()
lamb_dict[1] = [[0.6,0.4], [0.5,0.5], [0.4,0.6]]
for n in n_list:
    for k in k_list:
        for lamb_list in lamb_dict[n]:
            m = create_ngram_model(NgramModelWithInterpolation, 'train_e.txt', n, k)
            m.set_lambda(lamb_list)
            print('======')
            print('k is ', k)
            print('n is ', n)
            print('lambda is ', lamb_list)
            with open('val_e.txt', encoding='utf-8', errors='ignore') as f:
                print(m.perplexity(f.read()))
# m = create_ngram_model(NgramModelWithInterpolation, 'train_e.txt', 2, k=0.1)
# with open('val_e.txt', encoding='utf-8', errors='ignore') as f:
#     print(m.perplexity(f.read()))

k is  0.01
n is  1
lambda is  [0.6, 0.4]
1206.7958424364301
k is  0.01
n is  1
lambda is  [0.5, 0.5]
1240.0032530939195
k is  0.01
n is  1
lambda is  [0.4, 0.6]
1302.0066884501048
k is  0.001
n is  1
lambda is  [0.6, 0.4]
1030.2952286966201
k is  0.001
n is  1
lambda is  [0.5, 0.5]
1077.0983233651714
k is  0.001
n is  1
lambda is  [0.4, 0.6]
1152.302276439595
k is  0.005
n is  1
lambda is  [0.6, 0.4]
1128.5045820959924
k is  0.005
n is  1
lambda is  [0.5, 0.5]
1167.8842227570824
k is  0.005
n is  1
lambda is  [0.4, 0.6]
1235.489752926682
k is  0.01
n is  2
lambda is  [0.8, 0.1, 0.1]
2141.162987841561
k is  0.01
n is  2
lambda is  [0.5, 0.25, 0.25]
1356.8097030804897
k is  0.01
n is  2
lambda is  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
1196.313234737732
k is  0.01
n is  2
lambda is  [0.2, 0.5, 0.3]
1112.0706515982292
k is  0.001
n is  2
lambda is  [0.8, 0.1, 0.1]
1322.4320886823218
k is  0.001
n is  2
lambda is  [0.5, 0.25, 0.25]
910.0814321922466
k is  0.001
n is  

Please compare the perplexity of `shakespeare_sonnets.txt` when using word-level language model and character-level language model. In your writeup, explain why they are different .

### Aknowledgement:

This assigment is adapted from [Chris Callison-Burch's course CIS 530 - Computational Linguistics](http://computational-linguistics-class.org/index.html).